In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sn
import copy
import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader
import torchvision

# from sklearn import tree, svm
# from sklearn.pipeline import make_pipeline
# from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.model_selection import train_test_split

import seaborn as sn
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = .5
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (16, 10)

In [2]:
from lutils.fin.autoencoder import Normalizer

In [3]:
# np.set_printoptions(precision=8, suppress=True)

In [4]:
# %matplotlib inline
%matplotlib qt

In [5]:
n_epochs = 40 #100   # number of epochs to run
batch_size = 8  # size of each batch

In [6]:
class FutureTrendModule(nn.Module):
    def __init__(self):
        super(FutureTrendModule, self).__init__()
        
        input_size = 16
        hidden_size = 16
        num_layers = 4
        self.fc_size = hidden_size * 32
        
        # input batch_size 300
        self.fc1 = nn.Sequential(
            nn.Linear(1, input_size),
            nn.BatchNorm1d(32),
            nn.ReLU(),
        )
        
        self.lstm = nn.Sequential(
            nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        )
        
        self.dropout = nn.Dropout(0.2)
        
        # input batch_size 64
        self.output_fc = nn.Linear(self.fc_size, 1)
    
    def init_weights(self):
        for name, param in self.lstm.named_parameters():
            if 'bias' in name:
                nn.init.constant_(param, 0.0)
            elif 'weight_ih' in name:
                nn.init.kaiming_normal_(param)
            elif 'weight_hh' in name:
                nn.init.orthogonal_(param)
                    
    def forward(self, x): # 3 900
        batch_size = x.shape[0]
        
        fc1 = self.fc1(x)
        lstm_out, (h_n, c_n) = self.lstm(fc1)
        dropout = self.dropout(lstm_out.reshape(batch_size, -1))
        outs = self.output_fc(dropout)
        return outs, fc1, lstm_out, h_n, dropout

In [7]:
# class FutureTrendModule(nn.Module):
#     def __init__(self):
#         super(FutureTrendModule, self).__init__()
        
#         input_size = 16
#         hidden_size = 16
#         num_layers = 2
#         self.fc_size = hidden_size * num_layers
        
#         # input batch_size 300
#         self.fc1 = nn.Sequential(
#             nn.Linear(1, input_size),
#             nn.ReLU()
#         )
        
#         self.lstm = nn.Sequential(
#             nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
#         )
        
#         self.dropout = nn.Dropout(0.2)
        
#         # input batch_size 64
#         self.output_fc = nn.Linear(self.fc_size, 1)

#     def forward(self, x): # 3 900
#         batch_size = x.shape[0]
        
#         fc1 = self.fc1(x)
#         lstm1, (h_n, c_n) = self.lstm(fc1)
#         h_n = h_n.permute(1, 0, 2).reshape(-1, self.fc_size) # 8 2 16
#         dropout = self.dropout(h_n)
#         outs = self.output_fc(dropout)
#         return outs, fc1, lstm1, h_n, dropout

In [9]:
model = FutureTrendModule().to(0)
# model.load_state_dict(torch.load('D:/option/models/model_0612_01_l8_30S_5T_mean_h_n.pt'))
# model.load_state_dict(torch.load('D:/option/models/model_0618_lstm_diff_15S_15T.pt'))

<All keys matched successfully>

In [10]:
loss_fn = nn.MSELoss().to(0)  # mean square error
# loss_fn = nn.SmoothL1Loss().to(0)
# loss_fn = nn.CrossEntropyLoss().to(0)
# optimizer = optim.Adam(model.parameters(), lr=0.000001, betas=(0.9, 0.98), eps=1e-9)
# optimizer = optim.SGD(model.parameters(), lr = 0.000001, momentum=0.09)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1) #

optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)
lambda1 = lambda epoch: np.power(0.1, epoch) if epoch <= 6 else 1e-9
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)

In [11]:
# SAVE_PATH = 'D:/option/datas'
# SAVE_PATH = 'D:/option/l8'

# DATA_PATH = 'l8_1S_1T_mean'
# DATA_PATH = 'l8_10S_1T_mean'
# DATA_PATH = 'l8_30S_1T_mean'
# DATA_PATH = 'l8_30S_5T_mean'
# DATA_PATH = 'l8_60S_10T_mean'
# DATA_PATH = 'l8_30S_7T_mean'

# DATA_PATH = '1S_1T'
# DATA_PATH = '5S_1T'
# DATA_PATH = '10S_3T'
DATA_PATH = '15S_15T'
# DATA_PATH = '30S_15T'

In [12]:
# SAVE_PATH = os.path.join('D:/option/l8_autoencoder/', DATA_PATH)
# SAVE_PATH = os.path.join('D:/option/master/', DATA_PATH)

SAVE_PATH = os.path.join('D:/option/master/diff', DATA_PATH)

In [13]:
class TimeSeriesDataset(Dataset):
    def __init__(self, x, y):
        x = np.expand_dims(x, 2) # in our case, we have only 1 feature, so we need to convert `x` into [batch, sequence, features] for LSTM
        self.x = x.astype(np.float32)
        self.y = y.astype(np.float32)
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return (self.x[idx], self.y[idx])

In [116]:
writer = SummaryWriter('runs/log')

In [117]:
# Hold the best model
best_mse = np.inf   # init to infinity
best_weights = None
history = []

step = 0
# training loop
for epoch in range(n_epochs):

#     total_size = 0
    
#     for ix in range(0, len(dfs_x)-20):
    for j, f in enumerate(os.listdir(SAVE_PATH)):
        if not f.startswith('SHFE.rb'):
            continue
        
        store = pd.HDFStore(os.path.join(SAVE_PATH, f), 'r')
#         X = normalize(store['X'].values)
        X = store['X'].values
        y = store['y']['y'].values
        store.close()
        
        split_index = int(y.shape[0] * .8)
        
        data_x_train = X[:split_index]
        data_x_val = X[split_index:]
        data_y_train = y[:split_index]
        data_y_val = y[split_index:]
        
        dataset_train = TimeSeriesDataset(data_x_train, data_y_train)
        dataset_val = TimeSeriesDataset(data_x_val, data_y_val)
        
        train_dataloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
        
        model.train()
        total_size = data_y_train.shape[0]
        running_loss = 0.0
        for idx, (X_batch, y_batch) in enumerate(train_dataloader, 1):
            X_batch = X_batch.to(0)
            y_batch = y_batch.to(0)
            
            optimizer.zero_grad()
            
            y_pred, fc1, lstm1, h_n, dropout = model(X_batch)
            loss = loss_fn(y_pred.reshape(-1), y_batch)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * X_batch.shape[0]
            
            step += X_batch.shape[0]
            
#             if step % 8000 == 0:
            if idx == 1 and j % 4 == 0:
                writer.add_scalar('loss/loss', loss.item(), step)
                writer.add_scalar('loss/total_loss', running_loss / (total_size), step)

                writer.add_histogram('train/01_x', torchvision.utils.make_grid(X_batch).data.cpu(), step)
                writer.add_histogram('train/02_fc1', torchvision.utils.make_grid(fc1).data.cpu(), step)
                writer.add_histogram('train/03_lstm1', torchvision.utils.make_grid(lstm1).data.cpu(), step)
                writer.add_histogram('train/04_h_n', torchvision.utils.make_grid(h_n).data.cpu(), step)
                writer.add_histogram('train/05_dropout', torchvision.utils.make_grid(dropout).data.cpu(), step)

                writer.add_histogram('train/09_pred', torchvision.utils.make_grid(y_pred).data.cpu(), step)

#         print('Epoch: [{}/{}], [{}/{}], Loss: {:.8f}'.format(epoch + 1, n_epochs, start+10, total_size, running_loss / (X_batch.shape[0] * i)))
#         print('Epoch: [{}/{}], {}, Loss: {:.8f}'.format(epoch + 1, n_epochs, ix, running_loss / (total_size)))
#         print('Epoch: [{}/{}], {}, Loss: {:.8f}'.format(epoch + 1, n_epochs, f, running_loss / (total_size)))
        
        val_dataloader = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)
        
        model.eval()
        eval_loss = 0.0
        for idx, (X_batch, y_batch) in enumerate(val_dataloader, 1):
            X_batch = X_batch.to(0)
            y_batch = y_batch.to(0)
            
            y_pred, fc1, lstm1, h_n, dropout = model(X_batch)
            loss = loss_fn(y_pred.reshape(-1), y_batch).cpu().detach().numpy()
            
            eval_loss += loss.item() * y_batch.shape[0]
        eval_loss = eval_loss / data_y_val.shape[0]
        
#         print('eval loss {:.8f}'.format(eval_loss))
        lr = scheduler.get_last_lr()[0]
        print('Epoch: [{}/{}], {}, Loss: {:.8f}, Eval loss {:.8f}, lr:{:.16f}'.format(epoch + 1, n_epochs, f, running_loss / (total_size), eval_loss, lr))
        if eval_loss < best_mse:
            best_mse = eval_loss
            best_weights = copy.deepcopy(model.state_dict())
    
    scheduler.step()
    
# restore model and return best accuracy
model.load_state_dict(best_weights)

Epoch: [1/40], SHFE.rb.2021-04-06.15S.15T.20703.h5, Loss: 0.00294475, Eval loss 0.00175871, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2021-04-07.15S.15T.20703.h5, Loss: 0.00339007, Eval loss 0.00135700, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2021-04-08.15S.15T.20703.h5, Loss: 0.00269253, Eval loss 0.00133542, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2021-04-09.15S.15T.20703.h5, Loss: 0.00315724, Eval loss 0.00287804, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2021-04-12.15S.15T.20703.h5, Loss: 0.00330923, Eval loss 0.00126773, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2021-04-13.15S.15T.20703.h5, Loss: 0.00225106, Eval loss 0.00211351, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2021-04-14.15S.15T.20703.h5, Loss: 0.00176512, Eval loss 0.00120221, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2021-04-15.15S.15T.20703.h5, Loss: 0.00250898, Eval loss 0.00568236, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2021-04-16.15S.15T.20703.h5, Loss: 0.00232822, Eval loss 0.001193

Epoch: [1/40], SHFE.rb.2021-07-22.15S.15T.20703.h5, Loss: 0.00501282, Eval loss 0.00209336, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2021-07-23.15S.15T.20703.h5, Loss: 0.00363884, Eval loss 0.00261678, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2021-07-26.15S.15T.20703.h5, Loss: 0.00426635, Eval loss 0.00160861, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2021-07-27.15S.15T.20703.h5, Loss: 0.00485171, Eval loss 0.00317645, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2021-07-28.15S.15T.20703.h5, Loss: 0.00354409, Eval loss 0.00145708, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2021-07-29.15S.15T.20703.h5, Loss: 0.00326816, Eval loss 0.00275418, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2021-07-30.15S.15T.20703.h5, Loss: 0.00773924, Eval loss 0.01144689, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2021-08-02.15S.15T.20703.h5, Loss: 0.00665785, Eval loss 0.00305950, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2021-08-03.15S.15T.20703.h5, Loss: 0.00458303, Eval loss 0.003749

Epoch: [1/40], SHFE.rb.2021-11-10.15S.15T.20703.h5, Loss: 0.00827493, Eval loss 0.00461390, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2021-11-11.15S.15T.20703.h5, Loss: 0.00780903, Eval loss 0.00503549, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2021-11-12.15S.15T.20703.h5, Loss: 0.00625357, Eval loss 0.00389356, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2021-11-15.15S.15T.20703.h5, Loss: 0.00797198, Eval loss 0.00296524, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2021-11-16.15S.15T.20703.h5, Loss: 0.00483261, Eval loss 0.00353679, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2021-11-17.15S.15T.20703.h5, Loss: 0.00426764, Eval loss 0.00195439, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2021-11-18.15S.15T.20703.h5, Loss: 0.00354193, Eval loss 0.00399102, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2021-11-19.15S.15T.20703.h5, Loss: 0.00706709, Eval loss 0.00584291, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2021-11-22.15S.15T.20703.h5, Loss: 0.00502786, Eval loss 0.002159

Epoch: [1/40], SHFE.rb.2022-02-28.15S.15T.20703.h5, Loss: 0.00274173, Eval loss 0.00146595, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2022-03-01.15S.15T.20703.h5, Loss: 0.00275652, Eval loss 0.00115119, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2022-03-02.15S.15T.20703.h5, Loss: 0.00322411, Eval loss 0.00260117, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2022-03-03.15S.15T.20703.h5, Loss: 0.00423338, Eval loss 0.00249705, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2022-03-04.15S.15T.20703.h5, Loss: 0.00350809, Eval loss 0.00137330, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2022-03-07.15S.15T.20703.h5, Loss: 0.00722670, Eval loss 0.01049425, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2022-03-08.15S.15T.20703.h5, Loss: 0.00838768, Eval loss 0.00365956, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2022-03-09.15S.15T.20703.h5, Loss: 0.00583067, Eval loss 0.00198205, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2022-03-10.15S.15T.20703.h5, Loss: 0.00416568, Eval loss 0.002163

Epoch: [1/40], SHFE.rb.2022-06-16.15S.15T.20703.h5, Loss: 0.00279659, Eval loss 0.00473427, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2022-06-17.15S.15T.20703.h5, Loss: 0.00295600, Eval loss 0.00221506, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2022-06-20.15S.15T.20703.h5, Loss: 0.00726263, Eval loss 0.00423851, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2022-06-21.15S.15T.20703.h5, Loss: 0.00321371, Eval loss 0.00189347, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2022-06-22.15S.15T.20703.h5, Loss: 0.00394811, Eval loss 0.00204088, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2022-06-23.15S.15T.20703.h5, Loss: 0.00360636, Eval loss 0.00235077, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2022-06-24.15S.15T.20703.h5, Loss: 0.00386819, Eval loss 0.00544807, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2022-06-27.15S.15T.20703.h5, Loss: 0.00193866, Eval loss 0.00114139, lr:0.0000010000000000
Epoch: [1/40], SHFE.rb.2022-06-28.15S.15T.20703.h5, Loss: 0.00193469, Eval loss 0.001750

...

Epoch: [12/40], SHFE.rb.2022-11-03.15S.15T.20703.h5, Loss: 0.00133515, Eval loss 0.00077275, lr:0.0000000000010000
Epoch: [12/40], SHFE.rb.2022-11-04.15S.15T.20703.h5, Loss: 0.00124224, Eval loss 0.00092517, lr:0.0000000000010000
Epoch: [12/40], SHFE.rb.2022-11-07.15S.15T.20703.h5, Loss: 0.00136477, Eval loss 0.00077377, lr:0.0000000000010000
Epoch: [12/40], SHFE.rb.2022-11-08.15S.15T.20703.h5, Loss: 0.00111494, Eval loss 0.00090482, lr:0.0000000000010000
Epoch: [12/40], SHFE.rb.2022-11-09.15S.15T.20703.h5, Loss: 0.00107189, Eval loss 0.00101150, lr:0.0000000000010000
Epoch: [12/40], SHFE.rb.2022-11-10.15S.15T.20703.h5, Loss: 0.00148863, Eval loss 0.00081711, lr:0.0000000000010000
Epoch: [12/40], SHFE.rb.2022-11-11.15S.15T.20703.h5, Loss: 0.00128987, Eval loss 0.00088945, lr:0.0000000000010000
Epoch: [12/40], SHFE.rb.2022-11-14.15S.15T.20703.h5, Loss: 0.00122895, Eval loss 0.00083360, lr:0.0000000000010000
Epoch: [12/40], SHFE.rb.2022-11-15.15S.15T.20703.h5, Loss: 0.00100531, Eval loss

Epoch: [12/40], SHFE.rb.2023-02-21.15S.15T.20703.h5, Loss: 0.00092964, Eval loss 0.00060373, lr:0.0000000000010000
Epoch: [12/40], SHFE.rb.2023-02-22.15S.15T.20703.h5, Loss: 0.00122209, Eval loss 0.00124504, lr:0.0000000000010000
Epoch: [12/40], SHFE.rb.2023-02-23.15S.15T.20703.h5, Loss: 0.00155639, Eval loss 0.00086748, lr:0.0000000000010000
Epoch: [12/40], SHFE.rb.2023-02-24.15S.15T.20703.h5, Loss: 0.00129956, Eval loss 0.00126661, lr:0.0000000000010000
Epoch: [12/40], SHFE.rb.2023-02-27.15S.15T.20703.h5, Loss: 0.00154283, Eval loss 0.00085927, lr:0.0000000000010000
Epoch: [12/40], SHFE.rb.2023-02-28.15S.15T.20703.h5, Loss: 0.00105654, Eval loss 0.00046329, lr:0.0000000000010000
Epoch: [12/40], SHFE.rb.2023-03-01.15S.15T.20703.h5, Loss: 0.00100927, Eval loss 0.00042762, lr:0.0000000000010000
Epoch: [12/40], SHFE.rb.2023-03-02.15S.15T.20703.h5, Loss: 0.00092467, Eval loss 0.00057145, lr:0.0000000000010000
Epoch: [12/40], SHFE.rb.2023-03-03.15S.15T.20703.h5, Loss: 0.00080620, Eval loss

Epoch: [13/40], SHFE.rb.2021-05-14.15S.15T.20703.h5, Loss: 0.00906782, Eval loss 0.01042632, lr:0.0000000000010000
Epoch: [13/40], SHFE.rb.2021-05-17.15S.15T.20703.h5, Loss: 0.00807799, Eval loss 0.00286701, lr:0.0000000000010000
Epoch: [13/40], SHFE.rb.2021-05-18.15S.15T.20703.h5, Loss: 0.00483940, Eval loss 0.00357562, lr:0.0000000000010000
Epoch: [13/40], SHFE.rb.2021-05-19.15S.15T.20703.h5, Loss: 0.00976443, Eval loss 0.01453772, lr:0.0000000000010000
Epoch: [13/40], SHFE.rb.2021-05-20.15S.15T.20703.h5, Loss: 0.01058736, Eval loss 0.01130525, lr:0.0000000000010000
Epoch: [13/40], SHFE.rb.2021-05-21.15S.15T.20703.h5, Loss: 0.00647026, Eval loss 0.01360608, lr:0.0000000000010000
Epoch: [13/40], SHFE.rb.2021-05-24.15S.15T.20703.h5, Loss: 0.01291805, Eval loss 0.00445173, lr:0.0000000000010000
Epoch: [13/40], SHFE.rb.2021-05-25.15S.15T.20703.h5, Loss: 0.00512912, Eval loss 0.00504698, lr:0.0000000000010000
Epoch: [13/40], SHFE.rb.2021-05-26.15S.15T.20703.h5, Loss: 0.00710814, Eval loss

Epoch: [13/40], SHFE.rb.2021-08-25.15S.15T.20703.h5, Loss: 0.00347892, Eval loss 0.00266505, lr:0.0000000000010000
Epoch: [13/40], SHFE.rb.2021-08-26.15S.15T.20703.h5, Loss: 0.00477291, Eval loss 0.00330808, lr:0.0000000000010000
Epoch: [13/40], SHFE.rb.2021-08-27.15S.15T.20703.h5, Loss: 0.00317039, Eval loss 0.00456499, lr:0.0000000000010000
Epoch: [13/40], SHFE.rb.2021-08-30.15S.15T.20703.h5, Loss: 0.00389190, Eval loss 0.00223250, lr:0.0000000000010000
Epoch: [13/40], SHFE.rb.2021-08-31.15S.15T.20703.h5, Loss: 0.00345526, Eval loss 0.00239767, lr:0.0000000000010000
Epoch: [13/40], SHFE.rb.2021-09-01.15S.15T.20703.h5, Loss: 0.00427071, Eval loss 0.00224573, lr:0.0000000000010000
Epoch: [13/40], SHFE.rb.2021-09-02.15S.15T.20703.h5, Loss: 0.00649085, Eval loss 0.00305842, lr:0.0000000000010000
Epoch: [13/40], SHFE.rb.2021-09-03.15S.15T.20703.h5, Loss: 0.00433709, Eval loss 0.00200839, lr:0.0000000000010000
Epoch: [13/40], SHFE.rb.2021-09-06.15S.15T.20703.h5, Loss: 0.00448883, Eval loss

KeyboardInterrupt: 

In [118]:
model.load_state_dict(best_weights)

<All keys matched successfully>

In [119]:
y_pred

tensor([[ 0.0770],
        [ 0.3128],
        [ 0.2832],
        [-0.3135],
        [ 0.2455],
        [ 0.3146],
        [-0.4531],
        [ 0.0984]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [132]:
# torch.save(model.state_dict(), 'D:/option/models/model_0613_01_%s.pt' % DATA_PATH)
# torch.save(model.state_dict(), 'D:/option/models/model_0613_all_lstm4.pt')
# torch.save(model.state_dict(), 'D:/option/models/model_0616_all_lstm.pt')
# torch.save(model.state_dict(), 'D:/option/models/model_0618_lstm_diff.pt')
# torch.save(model.state_dict(), 'D:/option/models/model_0618_lstm_diff_15S_15T.pt')

In [121]:
# model.load_state_dict(torch.load('D:/option/models/model_0612_01_l8_30S_5T_mean_lstm4.pt'))

In [122]:
# model.load_state_dict(torch.load('D:/option/models/model_0613_all_lstm4.pt'))
# model.load_state_dict(torch.load('D:/option/models/model_0616_all_lstm.pt'))

In [123]:
# model.eval()
# model.training

In [124]:
# class Autoencoder(nn.Module):
#     def __init__(self): # 1 1
#         super(Autoencoder, self).__init__()
        
#         self.encoder = nn.Sequential(
#             nn.Linear(1, 128),
#             nn.BatchNorm1d(128),
#             nn.Sigmoid(),
#             nn.Linear(128, 64),
#             nn.Sigmoid(),
#             nn.Linear(64, 32),
#             nn.Sigmoid(),
#             nn.Linear(32, 1),
#         )
        
#         self.decoder = nn.Sequential(
#             nn.Linear(1, 32),
#             nn.Sigmoid(),
#             nn.Linear(32, 64),
#             nn.Sigmoid(),
#             nn.Linear(64, 128),
#             nn.Sigmoid(),
#             nn.BatchNorm1d(128),
#             nn.Linear(128, 1),
#         )
        
#     def forward(self, x):
#         encoded = self.encoder(x)
#         return self.decoder(encoded), encoded 

In [14]:
device = 0

In [15]:
# class Normalizer():
#     def __init__(self):
#         self.model = Autoencoder().to(device)
#         self.model.load_state_dict(torch.load('D:/option/models/SHFE.rb_encoder.pt'))

#     def fit_transform(self, x):
#         return self.model.encoder(torch.tensor(x, dtype=torch.float32, device=0).reshape([-1, 1])).cpu().detach().numpy()

#     def inverse_transform(self, x):
#         return self.model.decoder(torch.tensor(x, dtype=torch.float32, device=0).reshape([-1, 1])).cpu().detach().numpy()

# scaler = Normalizer('D:/option/models/SHFE.rb_encoder_256.pt', 0)

In [16]:
# TEST_DIR = os.path.join('D:/option/l8_autoencoder', '%s_test' % DATA_PATH)
TEST_DIR = os.path.join('D:/option/master/diff', '%s_test' % DATA_PATH)

In [17]:
def plot_file(p):
    store = pd.HDFStore(p, 'r')
    df_x = store['X']
    df_y = store['y']
    last_price = store['last_price']
    store.close()
    
    df_x = df_x.between_time('09:00', '10:15')
    df_y = df_y.between_time('09:00', '10:15')
    last_price = last_price.between_time('09:00', '10:15')
    
    X_test = df_x.values
    y_test = df_y['y'].values
    
    dataset_test = TimeSeriesDataset(X_test, y_test)
    test_dataloader = DataLoader(dataset_test, batch_size=batch_size)
    
    y_preds = []
    for idx, (X_batch, y_batch) in enumerate(test_dataloader, 1):
        y_pred, fc1, lstm1, h_n, dropout = model(X_batch.to(0))
        y_preds.append(y_pred.cpu().detach().numpy())
    
    y_pred = np.vstack(y_preds)
#     y_inverse = df_y['y'] * df_y['std'] + df_y['mu'] # (x*self.sd) + self.mu
#     y_inverse = scaler.inverse_transform(df_y['y'].values)
#     print(y_inverse)
#     print(last_price['last_price'].values)

    fig, ax1 = plt.subplots()

    ax1.plot(y_pred, label='pred', color='b', marker='x')
    ax1.plot(y_test, label='real', color='r', marker='.')

    ax2 = ax1.twinx()
    ax2.plot(last_price['last_price'].values, color='g', label='price', marker='1')
#     ax2.plot(y_inverse, color='y', label='inserse_price', marker='2')


    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc=0)

    fig.tight_layout()
    plt.show()

In [18]:
%matplotlib qt

In [19]:
# plot_file(os.path.join('D:/option/', 'SHFE_rb_mu.h5'))
# plot_file(os.path.join('D:/option/', 'SHFE_rb_mu_day.h5'))

# plot_file(os.path.join(TEST_DIR, 'SHFE.rb.2023-05-10.41384.h5'))
# plot_file(os.path.join(TEST_DIR, 'SHFE.rb.2023-05-04.30S.15T.20704.h5'))
# plot_file(os.path.join(TEST_DIR, 'SHFE.rb.2023-05-04.1S.1T.20704.h5'))
# plot_file(os.path.join(TEST_DIR, 'SHFE.rb.2023-05-04.5S.1T.20704.h5'))
plot_file(os.path.join(TEST_DIR, 'SHFE.rb.2023-05-04.15S.15T.20704.h5'))

In [38]:
# SHFE.rb.2023-05-04.41390.h5
# SHFE.rb.2023-05-10.41384.h5
store = pd.HDFStore(os.path.join(TEST_DIR, 'SHFE.rb.2023-05-10.41384.h5'), 'r')
df_x = store['X']
df_y = store['y']
last_price = store['last_price']
store.close()

OSError: ``D:\option\master\diff\1S_1T_test\SHFE.rb.2023-05-10.41384.h5`` does not exist

In [60]:
# X_test = normalize(df_x.values)
X_test = df_x.values
y_test = df_y['y'].values

In [61]:
dataset_test = TimeSeriesDataset(X_test, y_test)

In [62]:
test_dataloader = DataLoader(dataset_test, batch_size=batch_size)

In [63]:
y_preds = []
for idx, (X_batch, y_batch) in enumerate(test_dataloader, 1):
    y_pred, fc1, lstm1, h_n, dropout = model(X_batch.to(0))
    y_preds.append(y_pred.cpu().detach().numpy())

In [64]:
y_pred = np.vstack(y_preds)

In [65]:
y_inverse = df_y['y'] * df_y['std'] + df_y['mu'] # (x*self.sd) + self.mu

In [66]:
# plt.plot(y_pred, label='pred', marker='x')
# plt.plot(y_test, label='real', marker='.')
# plt.legend()
# plt.show()

In [67]:
%matplotlib qt

In [70]:
fig, ax1 = plt.subplots()

ax1.plot(y_pred, label='pred', color='b', marker='x')
ax1.plot(y_test, label='real', color='r', marker='.')

ax2 = ax1.twinx()
ax2.plot(last_price['last_price'].values, color='g', label='price', marker='1')
ax2.plot(y_inverse.values, color='y', label='inserse_price', marker='2')


lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=0)

fig.tight_layout()
plt.show()

In [71]:
%matplotlib inline

In [72]:
TEST_DIR = os.path.join('D:/option/', '%s_test' % DATA_PATH)

In [73]:
for f in os.listdir(TEST_DIR):
    print(f)
    store = pd.HDFStore(os.path.join(TEST_DIR, f), 'r')
    df_x = store['X']
    df_y = store['y']
    last_price = store['last_price']
    store.close()
    
    X_test = df_x.values
    y_test = df_y['y'].values
    y_inverse = df_y['y'] * df_y['std'] + df_y['mu']
    
    dataset_test = TimeSeriesDataset(X_test, y_test)
    test_dataloader = DataLoader(dataset_test, batch_size=batch_size)
    
    y_preds = []
    for idx, (X_batch, y_batch) in enumerate(test_dataloader, 1):
        y_pred, fc1, lstm1, h_n, dropout = model(X_batch.to(0))
        y_preds.append(y_pred.cpu().detach().numpy())
    
    y_pred = np.vstack(y_preds)
    
#     plt.plot(y_pred, label='pred')
#     plt.plot(y_test, label='real')
#     plt.legend()
#     plt.savefig(os.path.join('D:/option/', '%s_test_fig_lstm4' % DATA_PATH, '%s.jpg' % f), dpi=160)
#     plt.close()
    fig, ax1 = plt.subplots()

    ax1.plot(y_pred, label='pred', color='b', marker='x')
    ax1.plot(y_test, label='real', color='r', marker='.')

    ax2 = ax1.twinx()
    ax2.plot(last_price['last_price'].values, color='g', label='price', marker='1')
    ax2.plot(y_inverse.values, color='y', label='inserse_price', marker='2')
    
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc=0)

    fig.tight_layout()
    plt.savefig(os.path.join('D:/option/', '%s_test_fig' % DATA_PATH, '%s.jpg' % f), dpi=160)
    plt.close()
    

SHFE.rb.2023-05-04.41390.h5
SHFE.rb.2023-05-05.41398.h5
SHFE.rb.2023-05-08.41404.h5
SHFE.rb.2023-05-09.41399.h5
SHFE.rb.2023-05-10.41384.h5
SHFE.rb.2023-05-11.41406.h5
SHFE.rb.2023-05-12.41403.h5
SHFE.rb.2023-05-15.41392.h5
SHFE.rb.2023-05-16.41392.h5
SHFE.rb.2023-05-17.41389.h5
SHFE.rb.2023-05-18.41391.h5
SHFE.rb.2023-05-19.41398.h5
SHFE.rb.2023-05-22.41403.h5
SHFE.rb.2023-05-23.41404.h5
SHFE.rb.2023-05-24.41403.h5
SHFE.rb.2023-05-25.41407.h5
SHFE.rb.2023-05-26.41402.h5
SHFE.rb.2023-05-29.41378.h5
SHFE.rb.2023-05-30.41384.h5
SHFE.rb.2023-05-31.41393.h5
SHFE.rb.2023-06-01.41353.h5
SHFE.rb.2023-06-02.41400.h5
SHFE.rb.2023-06-05.41391.h5
SHFE.rb.2023-06-06.41401.h5
SHFE.rb.2023-06-07.41406.h5
SHFE.rb.2023-06-08.41400.h5
SHFE.rb.2023-06-09.41392.h5


### l8_30S_15T_mean 0.00028